In [2]:
from decouple import config
import facebook

In [3]:
page_token = config('PAGE_ACCESS_TOKEN')
page_id = config('PAGE_ID')

In [12]:
graph = facebook.GraphAPI(access_token=page_token, version="3.1")
print(graph)
page_info = graph.get_object(id=page_id, fields='about, website')
print(page_info)
posts = graph.get_object(id=page_id, fields='posts.limit(100)')
posts

{'id': '105141914536236'}


{'posts': {'data': [{'created_time': '2020-05-13T08:17:35+0000',
    'message': 'Lockdown can either be boring and sad, or it can be your chance to rest, have fun, and do all the stuff you previously had no time for. Any ideas for lockdown? Share some in the comments.',
    'id': '105141914536236_107376184312809'},
   {'created_time': '2020-05-13T08:14:16+0000',
    'message': 'These COVID-19 times will teach us the true power of saving. Yes, we have all heard that investing is better than saving, but this crisis has showed us that it is important to have some saved cash to fall back to. #Covid19 #Survival #Saving',
    'id': '105141914536236_107373617646399'},
   {'created_time': '2020-05-11T06:37:46+0000',
    'message': 'Welcome to Sanyu Means Joy!',
    'id': '105141914536236_105150181202076'}],
  'paging': {'cursors': {'before': 'QVFIUjU4Sk5ETkt5QTQ5MDZArR1VYTy1EenFJTVlMTDdfWDFrSWMtcGRvOTVYaWdIRGNSYk9lYTFXMjlpQUhUbnN4V0Ntc1dZAWXZAOSXhZAalVCV1BKWEVBejNhUkpSS25WWFRtdXZAaZAHFtRzRJYjV